In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import os

cwd = Path(os.getcwd())

ROOT_DIR_NAME = "ELITE-data-models"

for p in cwd.parents:
    if bool(re.search(ROOT_DIR_NAME + "$", str(p))):
        print(p)
        ROOT_DIR = p

/Users/nlee/Documents/Projects/ELITE/ELITE-data-models


In [2]:
from glob import glob

glob("data/RFC Tables/*.xlsx", root_dir=ROOT_DIR)

[]

In [3]:
other_values = ["Other", "Unknown", "Not collected", "Not applicable", "Not specified"]

In [4]:
new_template_path = "data/rfc_tables_raw/EL RFC genotyping_assay.xlsx"

template_name = Path(new_template_path)
template_name = re.sub("EL|RFC", "", template_name.stem).strip()
print(template_name)

new_temp_df = pd.read_excel(Path(ROOT_DIR, new_template_path))

new_temp_df

genotyping_assay


,key,description,valid values,required,requires,multivalue,type,concept source ontology,note
0,specimenID,Identifying string linked to a particular samp...,n/a (unique to each data contributor),1.0,"Biospecimen,\ngenotyping",0.0,string,Sage Bionetworks,NaN
1,sampleType,The type of sample collected,"Amniotic Fluid,\nAppendix,\nB cell,\nBasophils...",1.0,genotyping,0.0,string,"Sage Bionetworks,\nImmPort","The sample types are adopted from Uberon, Cell..."
2,specifySampleType,"If ""other"" is selected list the type of sample",NaN,0.0,"genotyping,\nsampleType = other",0.0,string,Sage Bionetworks,NaN
3,useReagent?,Was a reagent applied to the sample?,"Yes,\nNo",1.0,genotyping,0.0,boolean,"Sage Bionetworks,\nImmPort",NaN
4,reagentID(s),"One or more identifiers, separated by a semico...",n/a (unique to each data contributor),0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN
5,reagentName,The reagent name is an alternative to the Reag...,NaN,0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN
6,reagentManufacturer,The manufacturer is the source of a reagent an...,NaN,0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN
7,reagentCatalogNumber,"If the assay reagent is a commercial product, ...",NaN,0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN
8,reagentLotNumber,The lot number is often provided by a reagent ...,NaN,0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN
9,reagentWeblink,An internet address that may provide details o...,NaN,0.0,"genotyping,\nuseReagent = Yes",1.0,string,"Sage Bionetworks,\nImmPort",NaN


In [5]:
new_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   key                      22 non-null     object 
 1   description              22 non-null     object 
 2   valid values             11 non-null     object 
 3   required                 21 non-null     float64
 4   requires                 21 non-null     object 
 5   multivalue               21 non-null     float64
 6   type                     22 non-null     object 
 7   concept source ontology  21 non-null     object 
 8   note                     1 non-null      object 
dtypes: float64(2), object(7)
memory usage: 1.7+ KB


In [6]:
new_temp_df.isna().sum()

key                         0
description                 0
valid values               11
required                    1
requires                    1
multivalue                  1
type                        0
concept source ontology     1
note                       21
dtype: int64

# Cleanup

In [7]:
def remove_illegal_chars(x: str) -> str: 
    return re.sub("\(|\)|\?", "", x)

In [8]:
new_temp_df = new_temp_df.rename(
    columns={
        "key": "Attribute",
        "description": "Description",
        "required": "Required",
        "requires": "Module",
        "concept source ontology": "Ontology",
        "valid values": "Valid Values",
        "type": "columnType",
        "note": "Notes",
    }, errors='ignore'
)
new_temp_df = new_temp_df.fillna("")

In [9]:
# string clean up
# Do not need to explicitly add "other" type values. Will add back later for list type attributes
new_temp_df["Valid Values"] = (
    new_temp_df["Valid Values"]
    .replace("n/a (unique to each data contributor)", "", regex=False)
    .str.replace(
        "Other|Unknown|Not collected|Not applicable|Not specified", "", regex=True
    )
    .str.strip(",")
)

# replace note type values
new_temp_df["Valid Values"] = new_temp_df["Valid Values"].apply(
    lambda x: ",".join(
        [y.strip() for y in x.split(",") if not bool(re.search("Possible values", y))]
    ).strip(",")
)

new_temp_df = new_temp_df.replace(r"\n", ",", regex=True).replace(r",,", ",", regex=True)

# remove illegal characters from attributes
new_temp_df["Attribute"] = new_temp_df["Attribute"].apply(
    lambda x: re.sub("\(|\)|\?", "", x)
)

In [10]:
new_temp_df["Required"] = new_temp_df["Required"].astype(bool)

# new_temp_df["Required"] = (
#     new_temp_df["Required"]
#     .astype(str)
#     .apply(lambda x: False if bool(re.search("f", x)) else True)
#     .astype(bool)
# )

print(new_temp_df['Required'].unique())

[ True False]


In [11]:
new_temp_df = new_temp_df.drop(columns=["multivalue"], errors="ignore")

In [15]:
# split the values out and create new attributes to add to the model.
# These will be the intermediate attributes that will allow the original column i.e. SampleType to then have the user fill in specifySampleType
new_temp_df_others = new_temp_df.loc[new_temp_df["Module"].str.contains("=", na=False)].copy(deep=True)
new_temp_df_others["others"] = (
    new_temp_df_others["Module"]
    .str.split(",")
    .apply(lambda x: "".join([y.strip() for y in x if bool(re.search("=", y))]))
)

# in case there are multiple equals signs
new_temp_df_others = new_temp_df_others.explode("others")

new_temp_df_others["others"] = new_temp_df_others["others"].apply(remove_illegal_chars)

# set new parent values
new_temp_df_others["Parent"] = (
    new_temp_df_others["others"].str.split("=").apply(lambda x: x[0].strip())
)


# other value value used in valid values
new_temp_df_others["OtherValue"] = (
    new_temp_df_others["others"].str.split("=").apply(lambda x: x[1].strip())
)

new_temp_df_others["others"] = (
    new_temp_df_others["others"]
    .str.split("=")
    .apply(lambda x: x[1].strip().capitalize() + x[0][0].upper() + x[0][1:])
)

# Swap others -> Attribute and Attribute -> DependsOn
new_temp_df_others = new_temp_df_others.rename(
    columns={"Attribute": "DependsOn", "others": "Attribute"}
)

# hard coded values
new_temp_df_others["Required"] = False
new_temp_df_others["Module"] = "Other"
new_temp_df_others["Valid Values"] = ""

# Assign data modeling variables
new_temp_df_others = new_temp_df_others.assign(
    columnType="string",
    Ontology="Sage Bionetworks",
    Required=False,
    Properties="ValidValue",
)

for i in new_temp_df_others.index:
    new_temp_df_others.loc[i, "Description"] = (
        f"""When column = `{new_temp_df_others.loc[i, "OtherValue"]}`, add your custom value to the cell"""
    )

# remove illegal characters
new_temp_df_others[["Attribute", "Parent"]] = new_temp_df_others[
    ["Attribute", "Parent"]
].map(remove_illegal_chars)

new_temp_df_others

,Attribute,Description,Valid Values,Required,Module,columnType,Ontology,Notes,others,Parent,OtherValue
2,specifySampleType,"If ""other"" is selected list the type of sample",,False,"genotyping,sampleType = other",string,Sage Bionetworks,,OtherSampleType,sampleType,other
4,reagentIDs,"One or more identifiers, separated by a semico...",,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
5,reagentName,The reagent name is an alternative to the Reag...,,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
6,reagentManufacturer,The manufacturer is the source of a reagent an...,,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
7,reagentCatalogNumber,"If the assay reagent is a commercial product, ...",,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
8,reagentLotNumber,The lot number is often provided by a reagent ...,,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
9,reagentWeblink,An internet address that may provide details o...,,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
10,reagentContact,The contact information is particularly helpfu...,,False,"genotyping,useReagent = Yes",string,"Sage Bionetworks,ImmPort",,YesUseReagent,useReagent,Yes
12,treatmentIDs,"One or more identifiers, separated by a semico...",,False,"genotyping,useTreatment? = Yes",string,"Sage Bionetworks,ImmPort",,YesUseTreatment,useTreatment,Yes
13,treatmentName,Treatments refer to in vitro modifications of ...,,False,"genotyping,useTreatment? = Yes",string,"Sage Bionetworks,ImmPort",,YesUseTreatment,useTreatment,Yes


In [20]:
# Add other attributes to the list of valid values
new_temp_df.loc[
    ~new_temp_df.replace("", np.nan)["Valid Values"].isna(), "Valid Values"
] = new_temp_df.loc[~new_temp_df.replace("", np.nan)["Valid Values"].isna()].apply(
    lambda x: re.sub(
        ",+",
        ",",
        ",".join(
            [
                (
                    p
                    if p
                    not in new_temp_df_others.loc[
                        new_temp_df_others["Parent"] == x["Attribute"], "OtherValue"
                    ].values
                    else ""
                )
                for p in x["Valid Values"].split(",")
            ] + list(new_temp_df_others.loc[
                new_temp_df_others["Parent"] == x["Attribute"], "Attribute"
            ].values)
        ),
    ),
    axis=1,
)

# cleanup valid values
new_temp_df["Valid Values"] = (
    new_temp_df["Valid Values"]
    .str.split(",")
    .apply(lambda x: ",".join(list(np.unique(x))))
)

# Last bit of cleanup
new_temp_df["Properties"] = "ManifestColumn"
new_temp_df["Module"] = new_temp_df["Attribute"].apply(
    lambda x: "Other" if bool(re.search("specify", x)) else "Metadata"
)

new_temp_df

,Attribute,Description,Valid Values,Required,Module,columnType,Ontology,Notes,Properties
0,specimenID,Identifying string linked to a particular samp...,,True,Metadata,string,Sage Bionetworks,,ManifestColumn
1,sampleType,The type of sample collected,"Amniotic Fluid,Appendix,B cell,Basophils,Bone,...",True,Metadata,string,"Sage Bionetworks,ImmPort","The sample types are adopted from Uberon, Cell...",ManifestColumn
2,specifySampleType,"If ""other"" is selected list the type of sample",,False,Other,string,Sage Bionetworks,,ManifestColumn
3,useReagent,Was a reagent applied to the sample?,"No,YesUseReagent",True,Metadata,boolean,"Sage Bionetworks,ImmPort",,ManifestColumn
4,reagentIDs,"One or more identifiers, separated by a semico...",,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn
5,reagentName,The reagent name is an alternative to the Reag...,,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn
6,reagentManufacturer,The manufacturer is the source of a reagent an...,,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn
7,reagentCatalogNumber,"If the assay reagent is a commercial product, ...",,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn
8,reagentLotNumber,The lot number is often provided by a reagent ...,,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn
9,reagentWeblink,An internet address that may provide details o...,,False,Metadata,string,"Sage Bionetworks,ImmPort",,ManifestColumn


In [21]:
print("Shape of original data frame:", new_temp_df.shape)
print("Shape of others data frame:", new_temp_df_others.shape)

new_temp_df_final = pd.concat([new_temp_df, new_temp_df_others]).reset_index(drop=True)
new_temp_df_final = new_temp_df_final.set_index("Attribute")

new_temp_df_final['columnType'] = new_temp_df_final['columnType'].str.upper()

new_temp_df_final = new_temp_df_final.drop(columns=['OtherValue'])

new_temp_df_final = new_temp_df_final.reset_index().replace("", np.nan)

# combine duplicated rows
new_temp_df_final = new_temp_df_final.fillna("").astype(str).groupby('Attribute').apply(
    lambda x: x.apply(lambda y: ",".join(np.unique(y)))
)
new_temp_df_final = new_temp_df_final.drop(columns = 'Attribute')

print("Shape of final data frame:", new_temp_df_final.shape)

new_temp_df_final.info()

Shape of original data frame: (22, 9)
Shape of others data frame: (16, 12)
Shape of final data frame: (26, 10)
<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, OtherMeasurementTechnique  to useTreatment
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Description   26 non-null     object
 1   Valid Values  26 non-null     object
 2   Required      26 non-null     object
 3   Module        26 non-null     object
 4   columnType    26 non-null     object
 5   Ontology      26 non-null     object
 6   Notes         26 non-null     object
 7   Properties    26 non-null     object
 8   DependsOn     26 non-null     object
 9   Parent        26 non-null     object
dtypes: object(10)
memory usage: 2.2+ KB


/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_43240/4026748114.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_temp_df_final = new_temp_df_final.fillna("").astype(str).groupby('Attribute').apply(


In [22]:
new_temp_attr =[{
"Attribute":template_name ,
"Description":f"Template for {template_name}" ,
"DependsOn": 'Component,Filename,' + ','.join(list(new_temp_df_final.index)),
"Valid Values":"" ,
"Required":False ,
"Module":"Template" ,
"columnType":"" ,
"Ontology":"Sage Bionetworks" ,
"Notes":"" ,
"Properties":"" ,
"Parent":"Component",
}]

new_temp_attr

[{'Attribute': 'genotyping_assay',
  'Description': 'Template for genotyping_assay',
  'DependsOn': 'Component,Filename,OtherMeasurementTechnique ,OtherSampleType ,YesUseReagent ,YesUseTreatment ,measurementTechnique,reagentCatalogNumber,reagentContact,reagentIDs,reagentLotNumber,reagentManufacturer,reagentName,reagentWeblink,sampleType,specifyMeasurementTechnique,specifySampleType,specimenID,treatmentAmountUnit,treatmentAmountValue,treatmentDurationUnit,treatmentDurationValue,treatmentIDs,treatmentName,treatmentTemperatureUnit,treatmentTemperatureValue,useReagent,useTreatment',
  'Valid Values': '',
  'Required': False,
  'Module': 'Template',
  'columnType': '',
  'Ontology': 'Sage Bionetworks',
  'Notes': '',
  'Properties': '',
  'Parent': 'Component'}]

In [23]:
new_temp_df_final["UsedIn"] = template_name
new_temp_df_final = pd.concat([new_temp_df_final, pd.DataFrame(new_temp_attr).set_index('Attribute')]).replace("", np.nan)
new_temp_df_final

,Description,Valid Values,Required,Module,columnType,Ontology,Notes,Properties,DependsOn,Parent,UsedIn
Attribute,,,,,,,,,,,
OtherMeasurementTechnique,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifyMeasurementTechnique,measurementTechnique,genotyping_assay
OtherSampleType,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifySampleType,sampleType,genotyping_assay
YesUseReagent,"When column = `Yes`, add your custom value to ...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,"reagentCatalogNumber,reagentContact,reagentIDs...",useReagent,genotyping_assay
YesUseTreatment,"When column = `Yes`, add your custom value to ...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,"treatmentAmountUnit,treatmentAmountValue,treat...",useTreatment,genotyping_assay
measurementTechnique,The measurement technique describing the assay...,"16S rRNA gene sequencing,1D Gel,2D Gel,Array,B...",False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentCatalogNumber,"If the assay reagent is a commercial product, ...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentContact,The contact information is particularly helpfu...,NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentIDs,"One or more identifiers, separated by a semico...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentLotNumber,The lot number is often provided by a reagent ...,NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay


In [28]:
# unique index
new_temp_df_final.index.is_unique

True

In [29]:
new_temp_df_final = new_temp_df_final.sort_index(key = lambda x: x.str.lower())
new_temp_df_final

,Description,Valid Values,Required,Module,columnType,Ontology,Notes,Properties,DependsOn,Parent,UsedIn
Attribute,,,,,,,,,,,
genotyping_assay,Template for genotyping_assay,NaN,False,Template,NaN,Sage Bionetworks,NaN,NaN,"Component,Filename,OtherMeasurementTechnique ,...",Component,NaN
measurementTechnique,The measurement technique describing the assay...,"16S rRNA gene sequencing,1D Gel,2D Gel,Array,B...",False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
OtherMeasurementTechnique,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifyMeasurementTechnique,measurementTechnique,genotyping_assay
OtherSampleType,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifySampleType,sampleType,genotyping_assay
reagentCatalogNumber,"If the assay reagent is a commercial product, ...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentContact,The contact information is particularly helpfu...,NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentIDs,"One or more identifiers, separated by a semico...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentLotNumber,The lot number is often provided by a reagent ...,NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
reagentManufacturer,The manufacturer is the source of a reagent an...,NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay


In [30]:
# write out new cleaned template
new_temp_df_final.to_csv(Path(ROOT_DIR, 'data','rfc_tables_cleaned', template_name+'_cleaned_rfc.csv'))